# Clustering

## Introduction

Provided brief summary (1 to 2 paragraphs) about your feature data 
, and what you are trying to achieve with your clustering analysis.

## Theory

Write a brief technical write up about how EACH clustering method works (KMEAN, DBSAN, hierarchical clustering). Also include details on model selection methods that you use (elbow, silhouette, etc).
Don’t go “too deep”, around 2 to 4 paragraphs, per method, is fine. Write it in a way that a boss with a non-technical background would understand. Describe the method from a “big picture” point of view, how it works and what it is supposed to do.
Do this “in your own words”. 

### KMEAN

### DBSCAN

### Hierchical Clustering

## Methods

In this section, describe your coding workflow;
If you are not using Quarto, then provide links to the relevant code in this section.
If you are using Quarto (recommended), then include your code in-line using the “code folding” option, so that users can toggle the code on/off as needed. The notebook should also be hosted on Github. You can always do the entire thing in .ipynb then render it to HTML using Quarto at the end.

### Data Selection

If you have not done so already, create either a numeric record feature dataset 
 AND/OR a text feature dataset 
 from your existing data.
Remove the labels (targets) 
 as needed so that it is suitable for clustering.
If you have labels, you can use them at the end, to check if the clustering predictions coincided with the existing labels in the data-set. In general this may or may-not be true.
But these labels should not be used as part of the clustering analysis.

### Feature Selection

(optional)
If you want, you can perform filter based feature selection on your data-set 
 as a pre-processing step before clustering
You can also use optimal feature sets obtained during previous assignments


### Hyper-parameter tuning

For each of the three clustering algorithms, perform any relevant parameter tuning in an attempt to achieve the optimal clustering results
e.g. For k-means, Use Elbow and Silhouette methods to illustrate the ideal number of clusters. Visualize your results.
Also, when relevant, explore different choices of distance metric for the algorithm. Which distance metric seems to works best in which cases and why?

### FInal Results

Once you have everything “dialed in”, re-do the analysis one last time with the optimal parameter choice to get your “final results”.

## Results

Using your “final results”, discuss, illustrate, and compare the results of your various clustering analysis methods.
Which method seemed to work the best and why, which was easier to use or preferable, etc.
Can you make connections between the optimal cluster predictions, after parameter tuning, with any of the labels in the data set. Do they coincide? Why or why not?
Did the clustering results provide any new insights into your data?
Explore the results, and create as many meaningful visualizations as you need. Be creative, and experiment with different image aesthetics.
Ensure all visualizations are professional, ascetically pleasing, labeled, captioned, use color, are clear, and support your discussion and goals.

## Conslusions

In this section, the goal is to summarize & wrap-up the report. It explains what was found, in a way that would make sense to a general readership.
This area is non-technical. Technical descriptions of what you did should be in the methods or results sections, not conclusions.
The Conclusions should focus on key and important findings and how these findings affect real-life and real people.

## References

Reference all non-original content.
Ideally (but optionally) use .bibtex combined with Quarto to provide in-line internal citations
See the following link for an example:
https://drive.google.com/open?